In [1]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import os

def create_tunability_plots(model: str):
    df = pd.DataFrame(columns=['value', 'param', 'method'])

    # method = 'random_search' if sampling_method == 'random' else 'bayes_search'

    path = f'tunability/{model}'

    for folder in os.listdir(path):
        new_path = os.path.join(path, folder)
        for file in os.listdir(new_path):
            param = list(filter(lambda x: x not in [model, 'tunability.pickle'], file.split('_')))

            with open(os.path.join(new_path, file), 'rb') as f:
                tunability = pickle.load(f)
                df = pd.concat([df, pd.DataFrame({'value': tunability, 'param': '_'.join(param), 'method': folder})])

    if model == 'RFC':
        model_name = "Random Forest Classifier"
    elif model == 'XGB':
        model_name = "XGBoost Classifier"
    elif model == 'LR':
        model_name = "Logistic Regression"
    elif model == 'TREE':
        model_name = "Decision Tree Classifier"

    fig_path = f'plots/tunability/{model}'

    if not os.path.exists(fig_path):
        os.makedirs(fig_path)

    fig, ax = plt.subplots(1, 2, figsize=(20, 12))
    for i, method in enumerate(['random_search', 'bayes_search']):
        method_data = df[df['method'] == method]
        current_ax = ax[i]

        sns.boxplot(x='param', y='value', data=method_data, ax=current_ax, color='#235cc4')

        current_ax.set_title(f'Tunability for {model_name} using {method}')
        current_ax.set_ylabel('Tunability (AUC)')
        current_ax.set_xlabel('Hyperparameter')
        current_ax.set_xticklabels(current_ax.get_xticklabels(), rotation=45)

        if i == 1:
            current_ax.set_ylim(ax[0].get_ylim())
    plt.savefig(os.path.join(fig_path, f'{model}_tunability.png'), dpi=300, bbox_inches='tight')

In [ ]:
models = ['RFC', 'XGB', 'TREE']

for model in models:
    create_tunability_plots(model)